In [ ]:
import pandas as pd
from json_functions import create_df_big_output, create_example

players_list, df = create_df_big_output('very-big.json')

In [ ]:
import numpy as np
import tensorflow as tf

In [ ]:
features, outputs = df["rosters vector"], df['rating vector']

features = pd.DataFrame(features.values.tolist(), index= df.index)
outputs = pd.DataFrame(outputs.values.tolist(), index= df.index)

print(np.shape(features))
print(np.shape(outputs))

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy

In [ ]:
tf.config.list_physical_devices('GPU')

In [ ]:
# create loss function that only takes into account values from the output dont have a true value of 0

# This sigmoid is very close to a step function, if x = 0, then it will return nearly 0, and if x is a reasonable rating value,
# even something as low as 0.4, it will return (pretty much) 1. I will plug in y_true to this and then multiply the result by
# the error so that error on predictions where where y_true = 0 becomes tiny but where y_true is an actual rating stays 
# pretty much the same
def my_step(x):
    return tf.divide(1.0, tf.add(1.0, tf.exp(tf.multiply(-1000.0, tf.subtract(x, 0.02)))))

def my_loss(y_true, y_pred):
    # we can hardcode denominator as 10 since we know that there will always be 10 relevent values
    # in this case I just multiply by 0.1 for simplicity

    return tf.multiply(0.1, tf.reduce_sum(tf.multiply(my_step(y_true), tf.abs(tf.subtract(y_pred, y_true)))))

 

In [ ]:
# get the model
def get_model(n_inputs, n_outputs):
	model = Sequential()
	model.add(Dense(1068, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
	# model.add(Dense(50, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
	model.add(Dense(n_outputs))
	model.compile(loss=my_loss, optimizer='adam')
	return model

In [ ]:
n_inputs, n_outputs = features.shape[1], outputs.shape[1]

In [ ]:
model = get_model(n_inputs, n_outputs)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, outputs, test_size=0.33, random_state=42)

In [ ]:
model.fit(X_train, y_train, verbose=1, epochs=4, batch_size=500)
print("Evaluation on test set:")
mae = model.evaluate(X_test, y_test, verbose=1)

In [ ]:
example = create_example(['EliGE', 'Stewie2K', 'Grim', 'FalleN', 'NAF'], ['device', 'Xyp9x', 'Magisk', 'dupreeh', 'gla1ve'], players_list)
model.predict(np.asarray([example]))

In [ ]:
def print_prediction(example, players_list, model):
    player_indices = [i for i, x in enumerate(example) if x == 1]
    example_formatted = np.asarray([example])
    pred = model.predict(example_formatted)

    print("Team 1:")
    for i, player  in enumerate(player_indices):
        if player >= len(players_list): player =player - len(players_list)
        print(players_list[player]+': ', round(pred[0][player], 2))
        if i == 5:
            print("Agregate Rating:", round(np.sum(pred[0][0:5]), 2))
            print("Team 2:")
        if i == 9:
            print("Agregate Rating:", round(np.sum(pred[0][5:10]), 2))


In [ ]:
print_prediction(example, players_list, model)